In [ ]:
!pip install tensorflow -q
!pip install keras -q
!pip install keras_preprocessing -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import string, os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
from numpy.random import seed
seed(1)
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
df = pd.read_csv("/drive/My Drive/Colab Notebooks/dissertation/train.csv")
df['q_a'] = df['Questions'] + ' ' + df['Answers']
all_questions = list(df['q_a'])

def clean_text(txt):
    txt = "".join(t for t in txt if t not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt
corpus = [clean_text(x) for x in all_questions]
print(corpus[:10])

['how do i communicate datadriven insights to stakeholders within my startup clearly articulate the implications and recommendations based on the insights', 'how can i gather competitive intelligence and stay updated on market trends and innovations subscribe to industry newsletters and publications', 'how do i optimize my website for search engines seo to increase online visibility encourage social sharing and engagement to generate backlinks and increase visibility', 'how can i effectively monetize data or intellectual property assets in my startup offer data or ipbased services or solutions to customers', 'how do i create a userfriendly interface and intuitive navigation for the mobile app follow mobile app design guidelines and best practices', 'how can i gather reliable and trustworthy data for my startup data validation crossreference data from multiple sources and verify its accuracy to ensure reliability and minimize biases or errors', 'what techniques can i use to validate and

In [ ]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1

    ## convert data to a token sequence
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words
inp_sequences, total_words = get_sequence_of_tokens(corpus)
print(inp_sequences[:10])

[[6, 20], [6, 20, 3], [6, 20, 3, 99], [6, 20, 3, 99, 393], [6, 20, 3, 99, 393, 58], [6, 20, 3, 99, 393, 58, 2], [6, 20, 3, 99, 393, 58, 2, 300], [6, 20, 3, 99, 393, 58, 2, 300, 132], [6, 20, 3, 99, 393, 58, 2, 300, 132, 10], [6, 20, 3, 99, 393, 58, 2, 300, 132, 10, 15]]


In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [ ]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    # ----------Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    # ----------Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    # ----------Add Output Layer
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 426, 10)           33980     
                                                                 
 lstm (LSTM)                 (None, 100)               44400     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 3398)              343198    
                                                                 
Total params: 421,578
Trainable params: 421,578
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(predictors, label, epochs=10, verbose=5)

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list],              maxlen=max_sequence_len-1, padding='pre')
        predict_x=model.predict(token_list)
        predicted=np.argmax(predict_x,axis=1)
        #predicted = model.predict_classes(token_list, verbose=0)

        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [ ]:
test = pd.read_csv('/drive/My Drive/Colab Notebooks/dissertation/test.csv')

In [ ]:
test['lstm'] = test.apply(lambda x: generate_text(x['Questions'], 10, model, max_sequence_len), axis = 1)
test.to_csv('/drive/My Drive/Colab Notebooks/dissertation/test.csv',index=False)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 22ms/step


In [ ]:
print(generate_text("how do i communicate datadriven insights to stakeholders within my startup", 5, model, max_sequence_len))